In [1]:
# %matplotlib inline

In [5]:
# import matplotlib.pyplot as plt
# from matplotlib.animation import FuncAnimation
# from IPython import display
import numpy as np
import math
import time
import pygame

pygame 2.4.0 (SDL 2.26.4, Python 3.9.13)
Hello from the pygame community. https://www.pygame.org/contribute.html


In [1]:
class Body:
    #    pygame.draw.circle(win,color,(x,y),radius)
    G = 0.02 #`Gravitational` constant.Can be the real value or free choice
    signal_speed = 0.9
    def __init__(self, x,y,mass,radius = 0.2,color=(250,250,250)):
        self.x = x
        self.y = y
        self.mass = mass
        self.radius = radius
        self.color = color
        self.velocity = np.array([0,0])
        self.position_hystory = [[x,y]]
        self.signal_start_moments = [0]
        
    def __eq__(self, other):
        if self.mass == other.mass and self.radius == other.radius:
            return True
        return False
    
    @staticmethod
    def calc_acceleration(M,r):
        return Body.G * M/r**2

    @staticmethod
    def distance(point1,point2):
        return math.sqrt((point1[0] - point2[0]) ** 2 + (point1[1] - point2[1]) ** 2)
    
    @staticmethod
    def first_derivative(M,r):
        return 2 * Body.G * M/ r **3
    
    @staticmethod
    def escape_velocity(M,r):
        return math.sqrt((2 * M * Body.G)/ r)
    
    @staticmethod
    def vectors_calculations(body, other_object_x, other_object_y, other_object_mass):
           dist = body.distance([body.x, body.y], [other_object_x, other_object_y])
           dist_vector = [other_object_x - body.x, other_object_y - body.y]            
           first_deriv = body.first_derivative(other_object_mass, dist)    
           velocity_vector = np.array([dist_vector[0] * first_deriv, dist_vector[1] * first_deriv])    
           body.velocity = body.velocity + velocity_vector
           body.x += body.velocity[0]
           body.y += body.velocity[1]

    @staticmethod
    def calc_new_positions(frame,*bodies, newtonian_physics=True):
        for body in bodies:
            for other_body in bodies:
                if body != other_body:
                    if newtonian_physics:
                        
                         body.vectors_calculations(body,other_body.x,other_body.y,other_body.mass)
                    else:
                        
                        dist = body.distance([body.x,body.y], [other_body.position_hystory[0][0], other_body.position_hystory[0][1]])#                 
                        curr_time_interval = frame - other_body.signal_start_moments[0]               
                        signal_dist = dist - body.signal_speed * curr_time_interval
                        
                        if signal_dist <= 0:
                            other_body_x = other_body.position_hystory[0][0]
                            other_body_y = other_body.position_hystory[0][1]
                            other_body_mass = other_body.mass
                            
                            body.vectors_calculations(body,other_body_x,other_body_y,other_body_mass)
                            
                            del other_body.signal_start_moments[0]
                            del other_body.position_hystory[0]


In [1]:
import numpy as np
import math
import time
import pygame

class Body:
    #    pygame.draw.circle(win,color,(x,y),radius)
    G = 0.02 #`Gravitational` constant.Can be the real value or free choice
    signal_speed = 0.9
    def __init__(self, x,y,mass,radius = 0.2,color=(250,250,250)):
        self.x = x
        self.y = y
        self.mass = mass
        self.radius = radius
        self.color = color
        self.velocity = np.array([0,0])
        self.position_hystory = [[x,y]]
        self.signal_start_moments = [0]
        
    def __eq__(self, other):
        if self.mass == other.mass and self.radius == other.radius:
            return True
        return False
    
    @staticmethod
    def calc_acceleration(M,r):
        return Body.G * M/r**2

    @staticmethod
    def distance(point1,point2):
        return math.sqrt((point1[0] - point2[0]) ** 2 + (point1[1] - point2[1]) ** 2)
    
    @staticmethod
    def first_derivative(M,r):
        return 2 * Body.G * M/ r **3
    
    @staticmethod
    def escape_velocity(M,r):
        return math.sqrt((2 * M * Body.G)/ r)
    
    @staticmethod
    def vectors_calculations(body, other_object_x, other_object_y, other_object_mass):
           dist = body.distance([body.x, body.y], [other_object_x, other_object_y])
           dist_vector = [other_object_x - body.x, other_object_y - body.y]            
           first_deriv = body.first_derivative(other_object_mass, dist)    
           velocity_vector = np.array([dist_vector[0] * first_deriv, dist_vector[1] * first_deriv])    
           body.velocity = body.velocity + velocity_vector
           body.x += body.velocity[0]
           body.y += body.velocity[1]

    @staticmethod
    def calc_new_positions(frame,*bodies, newtonian_physics=True):
        for body in bodies:
            for other_body in bodies:
                if body != other_body:
                    if newtonian_physics:
                        
                         body.vectors_calculations(body,other_body.x,other_body.y,other_body.mass)
                    else:
                        
                        dist = body.distance([body.x,body.y], [other_body.position_hystory[0][0], other_body.position_hystory[0][1]])#                 
                        curr_time_interval = frame - other_body.signal_start_moments[0]               
                        signal_dist = dist - body.signal_speed * curr_time_interval
                        
                        if signal_dist <= 0:
                            other_body_x = other_body.position_hystory[0][0]
                            other_body_y = other_body.position_hystory[0][1]
                            other_body_mass = other_body.mass
                            
                            body.vectors_calculations(body,other_body_x,other_body_y,other_body_mass)
                            
                            del other_body.signal_start_moments[0]
                            del other_body.position_hystory[0]


pygame.init()
screen_width = 800
screen_height = 800
win = pygame.display.set_mode((screen_width, screen_height))
background_image = pygame.image.load('Star-Field.jpg')

circleA = Body(screen_width/2-100,screen_height/2,2000,radius= 10)

circleB = Body(screen_width/2+100,screen_height/2, 200, radius= 5,color=(250,0,0))

# circleC = Body(-10, -10, 40,radius = 0.4, color= 'blue')

circleA.velocity = np.array([0, 0])
circleB.velocity = np.array([0, -0.5])
# circleC.velocity = np.array([0.04, -0.02])

bodies = [circleA,circleB]



# Нарисуване на фоновото изображение на екрана
win.blit(background_image, (0, 0))
# Нарисуване на кръг
# circle_center = [screen_width/2, screen_width/2]
# radius = 10
# color = (250, 250,250)

clock = pygame.time.Clock()
FPS = 150  # желаните кадри в секунда
...
clock.tick(FPS)
# x,y = screen_width/2,screen_width/2
# z = 0
# speed_x = 0.01
# speed_y = -0.5

# Цикъл, който се изпълнява докато не бъде затворен прозореца
running = True
frame = 0
while running:
    frame += 1
    # Проверка за събития
    for event in pygame.event.get():
        if event.type == pygame.QUIT:
            running = False
    
    # Изпълнение на игрова логика
    # win.fill((0,0,0))
    win.blit(background_image, (0, 0))
    
    #Изчисляване на новите положения на всички участващи тела
    circleA.calc_new_positions(frame,*bodies, newtonian_physics=False)
    for body in bodies:
        # print(f'MAss: {body.mass} x: {body.x} y: {body.y}')
        pygame.draw.circle(win,body.color,(body.x,body.y),body.radius)
    # Опресняване на екрана

    clock.tick(FPS)
    pygame.display.update()
    pygame.display.flip()
# Изход от Pygame и Python
pygame.quit()
quit()






pygame 2.4.0 (SDL 2.26.4, Python 3.9.13)
Hello from the pygame community. https://www.pygame.org/contribute.html


IndexError: list index out of range